In [0]:
!mkdir dataset
from google.colab import files
src = list(files.upload().values())[0]
open('\dataset','wb').write(src)
!mv sample_mobile_data_1000.csv dataset/
!ls dataset

In [0]:
!curl ifconfig.co

34.76.163.135


In [64]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls /content/gdrive/My\ Drive/
!cp -r /content/gdrive/My\ Drive/BO-* /content/gdrive/My\ Drive/resume
!cd gdrive/My\ Drive/colab

 BO-DA-رزومه‌ها-E0		  nimac.gsite
 colab				  SahabX-Sokan
'Getting started.pdf'		 'اعضای دانشکده.xlsx'
'hadoop meeting feedback.gform'  'پروژه هوش سحاب.gdoc'
'Hadoop Meeting.gslides'	 'مشخصات سرور.gsheet'
 NimaBehrang.pdf


In [0]:
!unzip /content/gdrive/My\ Drive/colab/ml-100k.zip

In [0]:
!ls gdrive/My\ Drive/colab/ml-100k/

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


In [79]:
# !cat gdrive/My\ Drive/colab/ml-100k/u.user
import numpy as np
data = np.genfromtxt("/content/gdrive/My Drive/colab/ml-100k/u.data", dtype=int)
# data = np.genfromtxt("/content/gdrive/My Drive/colab/ml-100k/u.data",  dtype=[int,int,int,int])
train_data = []
test_data = []
users = np.unique(data[:,0])
for user in users:
  userrows = data[(np.where(data[:,0] == user)), :][0]
  np.random.shuffle(userrows)  
#   np.append(train_data, userrows[:20])
  for item in userrows[:20]:
    train_data.append(item.tolist())
  for item in userrows[20:]:
    test_data.append(item.tolist())

print(len(train_data))
print(np.shape(train_data))


18860
(18860, 4)


In [0]:
!ls 

In [0]:
!pip install -q tensorflow-gpu==2.0.0a0
!pip install -q tensorboard
!pip install -q tensorboardcolab

In [94]:
from tensorboardcolab import *
tbc=TensorBoardColab(port=6006, graph_path='/tmp/logs', startup_waiting_time=8)

Wait for 8 seconds...
TensorBoard link:
https://e658f22b.ngrok.io


In [0]:
from tensorflow.python import keras
import tensorflow as tf

In [0]:
keras.backend.clear_session()

In [0]:
input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
embed_user = keras.layers.Embedding(2000, 64)(input_user)
embed_item = keras.layers.Embedding(2000, 64)(input_item)
multiply = keras.layers.Multiply()([embed_user, embed_item])
rate = keras.layers.add(tf.unstack(multiply, axis=2))
model = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(rate))
# model.summary()

In [107]:
input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
embed_user = keras.layers.Embedding(2000, 64)(input_user)
embed_item = keras.layers.Embedding(2000, 64)(input_item)
dot = keras.layers.dot(inputs=[embed_user, embed_item],axes=2)
model_opt = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(dot))
model_opt.summary()
model_opt.compile(optimizer=tf.keras.optimizers.Adam(),
            loss='mean_squared_error',
            metrics=['accuracy','mae', 'AUC'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 64)        128000      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 64)        128000      input_4[0][0]                    
____________________________________________________________________________________________

In [109]:
train = np.asarray(train_data)
test = np.asarray(test_data)
x_train = [train[:,0],train[:,1]]
y_train = train[:,2]
x_test = [test[:,0],test[:,1]]
y_test = test[:,2]

logdir="/tmp/logs/2"
checkpoint_path = logdir + "/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=checkpoint_path)

# fit = model_opt.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10,
fit = model_opt.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30,
                callbacks = [cp_callback, tb_callback], use_multiprocessing=True, batch_size=1000)

ans = model_opt.evaluate(x_test, y_test)

Train on 18860 samples, validate on 81140 samples
Epoch 1/30
16000/18860 [========================>.....] - ETA: 0s - loss: 7.9636 - acc: 0.0574 - mean_absolute_error: 2.5875 - auc: 0.0000e+00
Epoch 00001: saving model to /tmp/logs/2/cp.ckpt
18860/18860 [==============================] - 0s 24us/sample - loss: 7.9884 - acc: 0.0570 - mean_absolute_error: 2.5922 - auc: 0.0000e+00 - val_loss: 7.7392 - val_acc: 0.0613 - val_mean_absolute_error: 2.5431 - val_auc: 0.0000e+00
Epoch 2/30
17000/18860 [==========================>...] - ETA: 0s - loss: 7.9733 - acc: 0.0568 - mean_absolute_error: 2.5902 - auc: 0.0000e+00
Epoch 00002: saving model to /tmp/logs/2/cp.ckpt
18860/18860 [==============================] - 0s 25us/sample - loss: 7.9872 - acc: 0.0570 - mean_absolute_error: 2.5922 - auc: 0.0000e+00 - val_loss: 7.7372 - val_acc: 0.0613 - val_mean_absolute_error: 2.5427 - val_auc: 0.0000e+00
Epoch 3/30
16000/18860 [========================>.....] - ETA: 0s - loss: 7.9715 - acc: 0.0587 - mean_

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item
